In [ ]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv('train.tsv', delimiter='\t')

In [ ]:
df.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [ ]:
df.dropna(inplace=True)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

df['processed_text'] = df['text_column'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyError: 'text_column'

In [ ]:
print(df.columns)

Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [ ]:
df = pd.read_csv('train.tsv', delimiter='\t')

In [ ]:
print(df.columns)

Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)


In [ ]:
df['processed_text'] = df['Phrase'].apply(preprocess_text)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_text'])

In [ ]:
y = df['Sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('logreg', LogisticRegression(max_iter=500))
])

In [ ]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('logreg', LogisticRegression(max_iter=500))])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('logreg', LogisticRegression(max_iter=1000, solver='saga'))
])

In [ ]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('logreg', LogisticRegression(max_iter=1000, solver='saga'))])

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('logreg', LogisticRegression(max_iter=2000, solver='liblinear'))
])

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('logreg',
                 LogisticRegression(max_iter=2000, solver='liblinear'))])

In [ ]:
y_pred = pipeline.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Accuracy: 0.6205305651672434
              precision    recall  f1-score   support

           0       0.45      0.32      0.37      1416
           1       0.52      0.42      0.46      5527
           2       0.69      0.82      0.75     15639
           3       0.54      0.48      0.51      6707
           4       0.52      0.34      0.41      1923

    accuracy                           0.62     31212
   macro avg       0.54      0.47      0.50     31212
weighted avg       0.60      0.62      0.61     31212



In [ ]:
#XG Boost Implementation
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import joblib



In [ ]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=5, eval_metric='mlogloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_xgb)}')
print(classification_report(y_test, y_pred_xgb))


Accuracy: 0.573753684480328
              precision    recall  f1-score   support

           0       0.62      0.15      0.24      1416
           1       0.53      0.14      0.22      5527
           2       0.58      0.95      0.72     15639
           3       0.55      0.25      0.34      6707
           4       0.63      0.19      0.29      1923

    accuracy                           0.57     31212
   macro avg       0.58      0.33      0.36     31212
weighted avg       0.57      0.57      0.50     31212



In [ ]:
y_pred_xgb = xgb_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_xgb)}')
print(classification_report(y_test, y_pred_xgb))


Accuracy: 0.573753684480328
              precision    recall  f1-score   support

           0       0.62      0.15      0.24      1416
           1       0.53      0.14      0.22      5527
           2       0.58      0.95      0.72     15639
           3       0.55      0.25      0.34      6707
           4       0.63      0.19      0.29      1923

    accuracy                           0.57     31212
   macro avg       0.58      0.33      0.36     31212
weighted avg       0.57      0.57      0.50     31212



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred_rf = rf_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_rf)}')
print(classification_report(y_test, y_pred_rf))

Accuracy: 0.6385364603357683
              precision    recall  f1-score   support

           0       0.49      0.39      0.43      1416
           1       0.55      0.49      0.52      5527
           2       0.71      0.80      0.76     15639
           3       0.55      0.50      0.53      6707
           4       0.49      0.40      0.44      1923

    accuracy                           0.64     31212
   macro avg       0.56      0.52      0.53     31212
weighted avg       0.63      0.64      0.63     31212

